
## graph based session intelligence

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("retailrocket/ecommerce-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/nahomnadew/.cache/kagglehub/datasets/retailrocket/ecommerce-dataset/versions/2


In [1]:
#initial parameters

DATA_PATH = "data/kaggle/events.csv"  
SAMPLE_MAX_SESSIONS = 200000  
MIN_SESSION_LENGTH = 2  
NEO4J_URI = "bolt://localhost:7687"
NEO4J_AUTH = ("neo4j", "nahi1420")  
BATCH_SIZE = 5000  


In [2]:
#importing necessary libraries
import os
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from tqdm import tqdm
from neo4j import GraphDatabase
import networkx as nx
from sklearn.model_selection import train_test_split

print("pandas", pd.__version__)

pandas 2.3.3


In [3]:
# Load dataset (may be large)
assert os.path.exists(DATA_PATH), f"Data file not found: {DATA_PATH}"
print("Loading, this may take a while for the full RetailRocket dataset...")

# The RetailRocket 'events.csv' has columns similar to: sessionId, itemId, eventType, timestamp
usecols = None  
df = pd.read_csv(DATA_PATH, parse_dates=['timestamp'], low_memory=False)
print("Loaded rows:", len(df))
print("Columns:", df.columns.tolist())
df.head()

Loading, this may take a while for the full RetailRocket dataset...


/tmp/ipykernel_8259/3092328460.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(DATA_PATH, parse_dates=['timestamp'], low_memory=False)


Loaded rows: 2756101
Columns: ['timestamp', 'visitorid', 'event', 'itemid', 'transactionid']


,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [6]:
# Corrected RetailRocket column mapping
cols = [c.lower() for c in df.columns]
colmap = {}
if 'visitorid' in cols:
    colmap[[c for c in df.columns if c.lower()=='visitorid'][0]] = 'session_id'
if 'itemid' in cols:
    colmap[[c for c in df.columns if c.lower()=='itemid'][0]] = 'event_id'
if 'event' in cols:
    colmap[[c for c in df.columns if c.lower()=='event'][0]] = 'event_type'
if 'timestamp' in cols:
    colmap[[c for c in df.columns if c.lower()=='timestamp'][0]] = 'timestamp'

df = df.rename(columns=colmap)
print('Renamed columns mapping:', colmap)

assert 'session_id' in df.columns and 'event_id' in df.columns and 'timestamp' in df.columns

# Add step_index per session (order by timestamp)
df = df.sort_values(['session_id','timestamp'])
df['step_index'] = df.groupby('session_id').cumcount() + 1

# Filter out sessions that are too short
session_lengths = df.groupby('session_id').size()
valid_sessions = session_lengths[session_lengths >= MIN_SESSION_LENGTH].index
df = df[df['session_id'].isin(valid_sessions)].copy()
print("After filtering short sessions:", df['session_id'].nunique(), "unique sessions,", len(df), "rows")

Renamed columns mapping: {'visitorid': 'session_id', 'event': 'event_type', 'timestamp': 'timestamp'}
After filtering short sessions: 406020 unique sessions, 1754541 rows


In [7]:
# pick the first N sessions (keeps temporal order inside sessions)
if SAMPLE_MAX_SESSIONS is not None:
    unique_sess = df['session_id'].drop_duplicates().iloc[:SAMPLE_MAX_SESSIONS]
    df = df[df['session_id'].isin(unique_sess)].copy()
    print("Sampled sessions:", df['session_id'].nunique(), "rows:", len(df))

Sampled sessions: 200000 rows: 869349


In [9]:
# Save a sampled CSV for quick checks 
sample_out = "data/sampled_sessions.csv"
df.to_csv(sample_out, index=False)
print("Saved sample to", sample_out)

Saved sample to data/sampled_sessions.csv


In [18]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

/tmp/ipykernel_8259/3748043458.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')


In [19]:
# Neo4j connection helper
def get_driver(uri=NEO4J_URI, auth=NEO4J_AUTH):
    return GraphDatabase.driver(uri, auth=auth, encrypted=False)

def create_constraints(driver):
    with driver.session() as session:
        session.execute_write(lambda tx: tx.run("""CREATE CONSTRAINT IF NOT EXISTS FOR (e:Event) REQUIRE e.id IS UNIQUE"""))
        session.execute_write(lambda tx: tx.run("""CREATE CONSTRAINT IF NOT EXISTS FOR (s:Session) REQUIRE s.id IS UNIQUE"""))
    print('Constraints ensured.')

def build_graph_from_df(df, driver, batch_size=BATCH_SIZE):
    # df must contain: session_id, timestamp, step_index, event_id, event_type (optional)
    create_constraints(driver)
    sessions = df['session_id'].drop_duplicates().tolist()
    print('Building nodes for', len(sessions), 'sessions (batched)')
    with driver.session() as session:
        for i in tqdm(range(0, len(sessions), batch_size), desc='sessions'):
            batch = sessions[i:i+batch_size]
            params = {'sids': batch}
            cypher = """UNWIND $sids AS sid
            MERGE (s:Session {id: sid})
            """
            session.run(cypher, **params)
    print('Creating Event nodes and OCCURRED_IN relationships (this may take a while)')
    with driver.session() as session:
        for sid, group in tqdm(df.groupby('session_id'), desc='sessions_events', total=df['session_id'].nunique()):
            g = group.sort_values('step_index')

            start_ts = g['timestamp'].iloc[0].isoformat() if not pd.isnull(g['timestamp'].iloc[0]) else None
            end_ts = g['timestamp'].iloc[-1].isoformat() if not pd.isnull(g['timestamp'].iloc[-1]) else None
            session.run("""MATCH (s:Session {id:$sid}) SET s.start_ts=$start_ts, s.end_ts=$end_ts, s.num_events=$num_events""", sid=sid, start_ts=start_ts, end_ts=end_ts, num_events=len(g))

            for r in g.itertuples():
                eid = str(r.event_id)
                etype = getattr(r, 'event_type', None) if 'event_type' in g.columns else None
                session.run("""MERGE (e:Event {id:$eid})
                ON CREATE SET e.type = $etype, e.global_count = 1
                ON MATCH SET e.global_count = coalesce(e.global_count,0) + 1
                """, eid=eid, etype=etype)
                session.run("""MATCH (e:Event {id:$eid}), (s:Session {id:$sid})
                MERGE (e)-[r:OCCURRED_IN {index:$idx, ts:$ts}]->(s)
                """, eid=eid, sid=sid, idx=int(r.step_index), ts=r.timestamp.isoformat() if not pd.isnull(r.timestamp) else None)
    print('Event nodes and OCCURRED_IN relationships created.')

def build_next_edges(df, driver):

    next_counts = Counter()
    for sid, group in df.groupby('session_id'):
        g = group.sort_values('step_index')
        events = list(g['event_id'].astype(str))
        for a,b in zip(events, events[1:]):
            next_counts[(a,b)] += 1
    print('Unique transitions:', len(next_counts))
    with driver.session() as session:
        for (a,b), cnt in tqdm(next_counts.items(), desc='creating_next'):
            session.run("""MATCH (a:Event {id:$a}), (b:Event {id:$b})
            MERGE (a)-[r:NEXT]->(b)
            ON CREATE SET r.count = $cnt
            ON MATCH SET r.count = coalesce(r.count,0) + $cnt
            """, a=a, b=b, cnt=int(cnt))
    print('NEXT edges created.')

In [20]:
driver = get_driver()
build_graph_from_df(df, driver)
build_next_edges(df, driver)
driver.close()
print('Graph build functions ready. To run: instantiate driver and call build_graph_from_df(...)')

Constraints ensured.
Building nodes for 200000 sessions (batched)


sessions:   0%|          | 0/40 [00:00<?, ?it/s]

sessions: 100%|██████████| 40/40 [00:02<00:00, 17.89it/s]


Creating Event nodes and OCCURRED_IN relationships (this may take a while)


sessions_events: 100%|██████████| 200000/200000 [1:29:49<00:00, 37.11it/s]   


Event nodes and OCCURRED_IN relationships created.
Unique transitions: 433803


creating_next: 100%|██████████| 433803/433803 [23:00<00:00, 314.25it/s]  

NEXT edges created.
Graph build functions ready. To run: instantiate driver and call build_graph_from_df(...)


In [21]:
def recommend_next_transition(driver, last_event_id, topk=10):
    q = """MATCH (e:Event {id:$last})-[r:NEXT]->(cand:Event)
    RETURN cand.id AS id, r.count AS score
    ORDER BY r.count DESC
    LIMIT $k
    """
    with driver.session() as s:
        res = s.run(q, last=last_event_id, k=topk)
        return [(r['id'], r['score']) for r in res]

# Example usage (uncomment after graph is built):
driver = get_driver()
print(recommend_next_transition(driver, "12345", topk=5))
driver.close()

[('92233', 1)]
